# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb659b22bb0>
├── 'a' --> tensor([[ 0.5489,  0.1182,  0.7261],
│                   [-1.4199, -3.3119,  2.1527]])
└── 'x' --> <FastTreeValue 0x7fb659b22f70>
    └── 'c' --> tensor([[ 0.2640, -0.4119,  0.1489,  0.0122],
                        [ 0.2020, -0.1225,  0.0267,  0.0761],
                        [-1.3395,  0.8177, -0.3104, -1.1202]])

In [4]:
t.a

tensor([[ 0.5489,  0.1182,  0.7261],
        [-1.4199, -3.3119,  2.1527]])

In [5]:
%timeit t.a

64.1 ns ± 0.0595 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb659b22bb0>
├── 'a' --> tensor([[ 0.3653,  0.4530,  0.2797],
│                   [-1.2025,  0.2704,  0.6294]])
└── 'x' --> <FastTreeValue 0x7fb659b22f70>
    └── 'c' --> tensor([[ 0.2640, -0.4119,  0.1489,  0.0122],
                        [ 0.2020, -0.1225,  0.0267,  0.0761],
                        [-1.3395,  0.8177, -0.3104, -1.1202]])

In [7]:
%timeit t.a = new_value

68.9 ns ± 0.0812 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5489,  0.1182,  0.7261],
               [-1.4199, -3.3119,  2.1527]]),
    x: Batch(
           c: tensor([[ 0.2640, -0.4119,  0.1489,  0.0122],
                      [ 0.2020, -0.1225,  0.0267,  0.0761],
                      [-1.3395,  0.8177, -0.3104, -1.1202]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5489,  0.1182,  0.7261],
        [-1.4199, -3.3119,  2.1527]])

In [11]:
%timeit b.a

63.4 ns ± 0.0127 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8133,  0.9091, -0.6794],
               [-1.1370, -0.5532,  0.9804]]),
    x: Batch(
           c: tensor([[ 0.2640, -0.4119,  0.1489,  0.0122],
                      [ 0.2020, -0.1225,  0.0267,  0.0761],
                      [-1.3395,  0.8177, -0.3104, -1.1202]]),
       ),
)

In [13]:
%timeit b.a = new_value

492 ns ± 0.161 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

805 ns ± 0.439 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 18.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 526 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 330 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb657b11c40>
├── 'a' --> tensor([[[ 0.5489,  0.1182,  0.7261],
│                    [-1.4199, -3.3119,  2.1527]],
│           
│                   [[ 0.5489,  0.1182,  0.7261],
│                    [-1.4199, -3.3119,  2.1527]],
│           
│                   [[ 0.5489,  0.1182,  0.7261],
│                    [-1.4199, -3.3119,  2.1527]],
│           
│                   [[ 0.5489,  0.1182,  0.7261],
│                    [-1.4199, -3.3119,  2.1527]],
│           
│                   [[ 0.5489,  0.1182,  0.7261],
│                    [-1.4199, -3.3119,  2.1527]],
│           
│                   [[ 0.5489,  0.1182,  0.7261],
│                    [-1.4199, -3.3119,  2.1527]],
│           
│                   [[ 0.5489,  0.1182,  0.7261],
│                    [-1.4199, -3.3119,  2.1527]],
│           
│                   [[ 0.5489,  0.1182,  0.7261],
│                    [-1.4199, -3.3119,  2.1527]]])
└── 'x' --> <FastTreeValue 0x7fb657b11e50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 42.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb581c06670>
├── 'a' --> tensor([[ 0.5489,  0.1182,  0.7261],
│                   [-1.4199, -3.3119,  2.1527],
│                   [ 0.5489,  0.1182,  0.7261],
│                   [-1.4199, -3.3119,  2.1527],
│                   [ 0.5489,  0.1182,  0.7261],
│                   [-1.4199, -3.3119,  2.1527],
│                   [ 0.5489,  0.1182,  0.7261],
│                   [-1.4199, -3.3119,  2.1527],
│                   [ 0.5489,  0.1182,  0.7261],
│                   [-1.4199, -3.3119,  2.1527],
│                   [ 0.5489,  0.1182,  0.7261],
│                   [-1.4199, -3.3119,  2.1527],
│                   [ 0.5489,  0.1182,  0.7261],
│                   [-1.4199, -3.3119,  2.1527],
│                   [ 0.5489,  0.1182,  0.7261],
│                   [-1.4199, -3.3119,  2.1527]])
└── 'x' --> <FastTreeValue 0x7fb581c06220>
    └── 'c' --> tensor([[ 0.2640, -0.4119,  0.1489,  0.0122],
                        [ 0.2020, -0.1225,  0.0267,  0.0761],
                 

In [23]:
%timeit t_cat(trees)

30.9 µs ± 57.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.3 µs ± 45.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2640, -0.4119,  0.1489,  0.0122],
                       [ 0.2020, -0.1225,  0.0267,  0.0761],
                       [-1.3395,  0.8177, -0.3104, -1.1202]],
              
                      [[ 0.2640, -0.4119,  0.1489,  0.0122],
                       [ 0.2020, -0.1225,  0.0267,  0.0761],
                       [-1.3395,  0.8177, -0.3104, -1.1202]],
              
                      [[ 0.2640, -0.4119,  0.1489,  0.0122],
                       [ 0.2020, -0.1225,  0.0267,  0.0761],
                       [-1.3395,  0.8177, -0.3104, -1.1202]],
              
                      [[ 0.2640, -0.4119,  0.1489,  0.0122],
                       [ 0.2020, -0.1225,  0.0267,  0.0761],
                       [-1.3395,  0.8177, -0.3104, -1.1202]],
              
                      [[ 0.2640, -0.4119,  0.1489,  0.0122],
                       [ 0.2020, -0.1225,  0.0267,  0.0761],
                       [-1.3395,  0.8177, -0.3104, -1.1202]],

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 695 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2640, -0.4119,  0.1489,  0.0122],
                      [ 0.2020, -0.1225,  0.0267,  0.0761],
                      [-1.3395,  0.8177, -0.3104, -1.1202],
                      [ 0.2640, -0.4119,  0.1489,  0.0122],
                      [ 0.2020, -0.1225,  0.0267,  0.0761],
                      [-1.3395,  0.8177, -0.3104, -1.1202],
                      [ 0.2640, -0.4119,  0.1489,  0.0122],
                      [ 0.2020, -0.1225,  0.0267,  0.0761],
                      [-1.3395,  0.8177, -0.3104, -1.1202],
                      [ 0.2640, -0.4119,  0.1489,  0.0122],
                      [ 0.2020, -0.1225,  0.0267,  0.0761],
                      [-1.3395,  0.8177, -0.3104, -1.1202],
                      [ 0.2640, -0.4119,  0.1489,  0.0122],
                      [ 0.2020, -0.1225,  0.0267,  0.0761],
                      [-1.3395,  0.8177, -0.3104, -1.1202],
                      [ 0.2640, -0.4119,  0.1489,  0.0122],
                   

In [28]:
%timeit Batch.cat(batches)

146 µs ± 357 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

338 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
